In [ ]:
pd.read_csv('DataFile/Reddit_Top_Submission.csv')

# Function#1 - Scraping No.1 Hot Submission Info in r/jobs :
- Subreddit ID
- Subreddit score
- Subreddit comment numbers
- Subreddit title
- Subreddit body

In [ ]:
def SubmInfo():
    import praw
    import pandas as pd

    reddit = praw.Reddit(
        client_id="bfx8ExsQXjGUptX6jLA5Rg",
        client_secret="fwnSOPiCXFTv4JtKSX9iqLd3r40YcA",
        user_agent="Scraper1",
    )

    hot_posts = reddit.subreddit('Jobs').hot()

    post_list = []

    for post in hot_posts:
        post_list.append([post.id, post.score, post.num_comments, post.title, post.selftext])


    post_df = pd.DataFrame(post_list, columns=['id','score','comm_num','title','body'])

    post_df = post_df.sort_values('score',ascending=False).reset_index()
    post_df.drop(['index'], axis=1, inplace=True)
    
    top_subm_dict = post_df.iloc[0].to_dict()

    return top_subm_dict

# Function#2 - Getting the top comments + Extensive cleaning :
- Lower-casing
- Tokenization + Removing punctuations
- Removing stopwords
- Lemmatization

In [ ]:
def TopComm(TopSubmInfoDF):
    import praw
    from praw.models import MoreComments

    import nltk
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer


    import string

    import pandas as pd
    import re

    reddit = praw.Reddit(
            client_id="bfx8ExsQXjGUptX6jLA5Rg",
            client_secret="fwnSOPiCXFTv4JtKSX9iqLd3r40YcA",
            user_agent="Scraper1",
            )

    top_id = TopSubmInfoDF['id']
    submission = reddit.submission(id=top_id)

    comment_list = []

    submission.comments.replace_more(limit=0)
    for comm in submission.comments:
        
        #1 - Lower-casing
        comment = comm.body.lower()

        #2 - Tokenize and removing punctuations
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        clean = tokenizer.tokenize(comment)

        #3 - Removing stopwords and singular characters
        stopword = stopwords.words('english')
        stopword = stopword + list(string.ascii_lowercase)
        word_list = []
        for word in clean:
            if word not in stopword:
                word_list.append(word)

        #4 - Lemmatization
        lemmatizer = WordNetLemmatizer()
        for w in word_list:
            comment_list.append(lemmatizer.lemmatize(w))

    return comment_list

### Mini Checkpoint : Run Scrapers and return CommList for Plotting

In [ ]:
comment_list = rscraper2(rscraper1())

# Function#3 - Printing Word Frequency Line Graph

In [ ]:
def rplot1(comment_list):
    from nltk.probability import FreqDist
    import matplotlib as pl
    %matplotlib inline
    import matplotlib.pyplot as plt

    fdist=FreqDist(comment_list)
    fdist.plot(20,cumulative=False)

In [ ]:
rplot1(comment_list)

# Fucntion#4 - Printing WordCloud

In [ ]:
def rplot2(comment_list):
    from wordcloud import WordCloud, ImageColorGenerator

    import matplotlib as pl
    %matplotlib inline
    import matplotlib.pyplot as plt
    
    import numpy as np
    
    from PIL import Image
    
    # Because WordCloud takes in string but not list
    comment_string = ' '.join(comment_list)
    
    # Mask with a RM50 bill
    mask = np.array(Image.open('./RM100.jpg')) 
    color= ImageColorGenerator(mask)
    
    # WordCloud settings
    wordcloud=WordCloud(
        background_color='white', 
        max_font_size=200,
        max_words=1000,
        mask=mask,
        random_state=42,
        repeat=True,
        )
    
    # WordCloud generator
    wordcloud.generate(comment_string)
    
    # WordCloud plotter
    plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(wordcloud.recolor(color_func=color), interpolation="bilinear")
    plt.show()

In [ ]:
rplot2(comment_list)

# Function#5 - A repeating cron job tasked everyday to scrape new info :
- If the new top hot submission has a higher score than previous, output a new wordcloud
- If the existing top hot submission is still the higher score, update the word counts and reprint wordcloud.

# Main Function

In [ ]:
#IMPORT_ANTS#
import praw
from praw.models import MoreComments

import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import string

import os.path

#INITIATING REDDIT
reddit = praw.Reddit(
    client_id="bfx8ExsQXjGUptX6jLA5Rg",
    client_secret="fwnSOPiCXFTv4JtKSX9iqLd3r40YcA",
    user_agent="Scraper1",
)

#GETTING HOT POST
hot_posts = reddit.subreddit('Jobs').hot()

post_list = []

for post in hot_posts:
    post_list.append([post.id, post.score, post.num_comments, post.title, post.selftext])


post_df = pd.DataFrame(post_list, columns=['id','score','comm_num','title','body'])

post_df = post_df.sort_values('score',ascending=False).reset_index()
post_df.drop(['index'], axis=1, inplace=True)

top_subm_id = post_df['id'][0]
submission = reddit.submission(id=top_subm_id)

word_list = []
sent_list = []

submission.comments.replace_more(limit=0)
for comm in submission.comments:
    print(comm.body)

In [ ]:
#IMPORT_ANTS#
import praw
from praw.models import MoreComments

import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import string

import os.path

from datetime import date

#INITIATING REDDIT
reddit = praw.Reddit(
    client_id="bfx8ExsQXjGUptX6jLA5Rg",
    client_secret="fwnSOPiCXFTv4JtKSX9iqLd3r40YcA",
    user_agent="Scraper1",
)

#GETTING HOT POST
hot_posts = reddit.subreddit('Jobs').hot()

post_list = []

for post in hot_posts:
    post_list.append([post.id, post.score, post.num_comments, post.title, post.selftext])


post_df = pd.DataFrame(post_list, columns=['id','score','comm_num','title','body'])

post_df = post_df.sort_values('score',ascending=False).reset_index()
post_df.drop(['index'], axis=1, inplace=True)

top_subm_id = post_df['id'][0]
submission = reddit.submission(id=top_subm_id)

word_list = []
sent_list = []

submission.comments.replace_more(limit=0)
for comm in submission.comments:
    
    #1 - Lower-casing
    comment = comm.body.lower()
    
    #2 - Tokenize and removing punctuations
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    clean_word = tokenizer.tokenize(comment)
    
    #3 - Removing singular characters and returning sentence
    words_wout_alphabets = []
    for w in clean_word:
        if w not in list(string.ascii_lowercase):
            words_wout_alphabets.append(w)
    clean_sent = ' '.join(words_wout_alphabets)
    sent_list.append(clean_sent)
    
    #4 - Removing stopwords
    stopword = stopwords.words('english')
    words_wout_stopwords = []
    for w in words_wout_alphabets:
        if w not in stopword:
            words_wout_stopwords.append(w)
    
    #5 - Lemmatization
    lemmatizer = WordNetLemmatizer()
    for w in words_wout_stopwords:
        word_list.append(lemmatizer.lemmatize(w))

Subm_Today = {
    "date" : str(date.today()),
    "id" : post_df['id'][0],
    "score" : int(post_df['score'][0]),
    "title" : post_df['title'][0],
    "body" : post_df['body'][0],
    "sent_num" : len(sent_list),
    "sent_str" : str(sent_list),
    "word_num" : len(word_list),
    "u_word_num" : len(set(word_list)),
    "word_str" : str(word_list),
}

to_save = pd.DataFrame(Subm_Today,index=[0])

if  os.path.isfile('DataFile/Reddit_Top_Submission.csv'):
    df = pd.read_csv('DataFile/Reddit_Top_Submission.csv')
    df_updated = pd.concat([df,to_save])
else:
    df_updated = to_save
df_updated.to_csv('DataFile/Reddit_Top_Submission.csv',index=False)